In [1]:
!pip install langchain-community langchain-core langchain-openai langchain-Chroma beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.5 MB/s eta 0:00:0

In [ ]:
import os
os.environ['OPENAI_API_KEY']="sk-proj-"
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = "lsv2_pt"
os.environ['LANGCHAIN_PROJECT'] = "memory_rag"

In [39]:
import bs4
from langchain import hub
from langchain_openai import ChatOpenAI , OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory   # the entire conservation is managed inside the list
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

In [16]:
llm_model = ChatOpenAI(model= 'gpt-3.5-turbo')
embedding_model = OpenAIEmbeddings()
parser = StrOutputParser()

In [5]:
while True:
  message = input("Write your query:")
  if message=="bye":
    print("Bye have a great day!")
    break
  else:
    print(parser.invoke(llm_model.invoke([HumanMessage(content=message)])))

Write your query:hi
Hello! How can I assist you today?
Write your query:bye
Bye have a great day!


# Collecting the data

In [8]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only = bs4.SoupStrainer(class_=("post-content","post-title",'post-header'))
    ),
)

In [9]:
doc = loader.load()

In [10]:
len(doc)

1

# Splitting the text

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 200,
)

In [12]:
splits = text_splitter.split_documents(doc)

In [14]:
splits[0].page_content

'LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory'

In [15]:
len(splits)

63

# Creating the VectorStore

In [17]:
vectorstores = Chroma.from_documents(splits, embedding_model) # keeping the memory in memory not persistent
retriever = vectorstores.as_retriever()

In [18]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7aa112f96b70>, search_kwargs={})

In [46]:
system_prompt = (
    """
    you are an assistant for question and answering tasks.
    use the following pieces of context to answer the question at the end.
    if you don't know the answer, just say that you don't know, don't try to make up an answer.
    Use three sentences meximum and keep the answer consise.
    \n\n
    ----------------
    {context}
    ----------------
    """
)

In [47]:
prompt = ChatPromptTemplate.from_messages(
    [
    ('system',system_prompt),
    ('human','{input}'),
])


In [48]:
question_answering_chain = create_stuff_documents_chain(llm_model , prompt)

In [50]:
question_answering_chain.invoke({'input': 'What is MRKL ?', 'context': []})

"I'm not sure what MRKL stands for."

In [34]:
rag_chain = create_retrieval_chain(retriever , question_answering_chain)

In [35]:
response = rag_chain.invoke({'input': 'What is MRKL ?'})

In [38]:
response['answer']

'MRKL stands for "Modular Reasoning, Knowledge, and Language", which is a neuro-symbolic architecture for autonomous agents proposed by Karpas et al. in 2022. It consists of expert modules and a general-purpose LLM that routes inquiries to the best suitable expert module, which can be neural or symbolic in nature. These modules, such as deep learning models or calculators, enhance the capabilities of the autonomous agents by providing specialized functionalities.'

# Adding Memory component Manually


In [51]:
from langchain.chains import create_history_aware_retriever

In [52]:
retriever_prompt = (
    """
    Given a chat history and the latest user question which might reference context in the chat history,
    formulate the standalone question which can be understood without the chat histoy.
    Do not answer the quesiton, just reformulate it if needed and otherwise return it as is.
    """
)

In [55]:
contxtualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ('system',retriever_prompt),
        MessagesPlaceholder(variable_name='chat_history'), # past message will be inserted automatically when the chain runs
        ('human','{input}'),
    ]
)   # This chain is for the retriever only

In [56]:
history_aware_retriever = create_history_aware_retriever(llm_model , retriever , contxtualize_q_prompt | llm_model | StrOutputParser())

In [57]:
qa_prompt = ChatPromptTemplate.from_messages([
    ('system',system_prompt),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human','{input}'),
]) # this chain is for the LLM for the ouput

In [58]:
question_answer_chain = create_stuff_documents_chain(llm_model , qa_prompt)

In [59]:
asnwer_chain = create_retrieval_chain(history_aware_retriever , question_answer_chain)

In [60]:
chat_history = []

In [61]:
question1 = "What is task Decomposition"

In [62]:
message1 = asnwer_chain.invoke({'input': question1 , 'chat_history': chat_history})

In [64]:
message1['answer']

'Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This allows models or agents to better tackle difficult problems by dividing them into more manageable subtasks. Different methodologies like Chain of Thought and Tree of Thoughts guide the process of task decomposition to enhance model performance on complex tasks.'

In [65]:
chat_history.extend([HumanMessage(content=question1) , AIMessage(content=message1['answer'])])

In [66]:
chat_history

[HumanMessage(content='What is task Decomposition', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This allows models or agents to better tackle difficult problems by dividing them into more manageable subtasks. Different methodologies like Chain of Thought and Tree of Thoughts guide the process of task decomposition to enhance model performance on complex tasks.', additional_kwargs={}, response_metadata={})]

In [67]:
second_question = "What are the common ways of doing it ?"
message2 = asnwer_chain.invoke({'input': second_question , 'chat_history': chat_history})

In [69]:
print(message2['answer'])

Task decomposition can be achieved in several common ways:
1. Language Model with simple prompting, such as asking for steps or subgoals.
2. Task-specific instructions tailored to the nature of the task, like outlining a story for writing a novel.
3. Outsourcing the planning step to a classical planner using the LLM+P approach, which involves translating the problem and generating plans with the help of Planning Domain Definition Language (PDDL).


# Adding the Memory automatically

In [70]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [71]:
store = {}

In [78]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [79]:
conversational_rag_chain = RunnableWithMessageHistory(
    asnwer_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)


In [80]:

conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]


'Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This approach helps to manage and solve challenging problems by dividing them into more manageable subtasks. Different methods, such as Chain of Thought and Tree of Thoughts, can be employed to decompose tasks effectively.'

In [83]:
store


{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This approach helps to manage and solve challenging problems by dividing them into more manageable subtasks. Different methods, such as Chain of Thought and Tree of Thoughts, can be employed to decompose tasks effectively.', additional_kwargs={}, response_metadata={})])}

In [84]:

conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]


'Task decomposition can be achieved in several common ways:\n1. Using language models with simple prompts like "Steps for XYZ" or "What are the subgoals for achieving XYZ?"\n2. Providing task-specific instructions tailored to the nature of the task, for example, "Write a story outline" for novel writing.\n3. Involving human inputs to guide the decomposition process effectively.'

In [85]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This approach helps to manage and solve challenging problems by dividing them into more manageable subtasks. Different methods, such as Chain of Thought and Tree of Thoughts, can be employed to decompose tasks effectively.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are common ways of doing it?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition can be achieved in several common ways:\n1. Using language models with simple prompts like "Steps for XYZ" or "What are the subgoals for achieving XYZ?"\n2. Providing task-specific instructions tailored to the nature of the task, for example, "Write a story outline" for novel writing.\n3. Involving human inputs to guide the 

In [86]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This approach helps to manage and solve challenging problems by dividing them into more manageable subtasks. Different methods, such as Chain of Thought and Tree of Thoughts, can be employed to decompose tasks effectively.

User: What are common ways of doing it?

AI: Task decomposition can be achieved in several common ways:
1. Using language models with simple prompts like "Steps for XYZ" or "What are the subgoals for achieving XYZ?"
2. Providing task-specific instructions tailored to the nature of the task, for example, "Write a story outline" for novel writing.
3. Involving human inputs to guide the decomposition process effectively.

